In [ ]:
import nibabel as nib
import nipy as nipy
import matplotlib.pyplot as plt
import nipype.algorithms.rapidart as rpd    # rapidart module
import nipype.interfaces.io as nio          # Data i/o
import nipype.interfaces.fsl as fsl         # fsl
import nipype.interfaces.utility as util    # utility
import nipype.pipeline.engine as pe         # pypeline engine
import numpy as np
import os, sys
from tqdm import tqdm
import time

In [ ]:
def getSimilarityBetweenVolumes(source, target, scheduleTXT):
    import subprocess
    import nipype.interfaces.fsl as fsl  # fsl
    
    #building the command
    flirt = fsl.FLIRT()
    flirt.inputs.in_file   = source
    flirt.inputs.cost      = 'corratio'
    flirt.inputs.reference = target
    flirt.inputs.schedule  = scheduleTXT

    #running the command
    cmdline = flirt.cmdline.split(' ')
    result = subprocess.run(cmdline, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
    
    #make sure there was no errors
    if result.returncode != 0:
        print(f"Error occurred: {result.stderr}")
        return None
    
    #capture and return the similarity as a float. 
    return float(result.stdout.split()[0])

In [ ]:
# Extracts and returns brain from skull image
def getBET(input_file, fraction):
    fsl_bet = fsl.BET(in_file=input_file, frac=fraction)
    output = fsl_bet.run()
    return output.outputs.out_file

In [ ]:
# Extracts and returns brain mask from skull image
# Output file name check
def getMask(input_file, mask_file):
    fsl_mask = fsl.ApplyMask(in_file=input_file, mask_file=mask_file)
    output=fsl_mask.run()
    return output.outputs.out_file

In [ ]:
# Registers input image onto the reference image
def getFLIRT(input_file, reference_file):
    fslflirt = fsl.FLIRT()
    fslflirt.inputs.in_file = input_file
    fslflirt.inputs.reference = reference_file
    output = fslflirt.run()
    return output.outputs.out_file

In [ ]:
# Sorts data into dictionary and returns dictionary
def findFileLocations():
    subjDict = {}
    dataDir = "/data/subjects"
    for subj in os.listdir(dataDir):
        if subj.isdigit():
            fetalFol = subj + "_fetal"
            neonateFol = subj + "_neonate" 
            fetalDir = os.path.join(dataDir, subj, fetalFol)
            fetalMask = None
            fetalTemplate = None
            for file in os.listdir(fetalDir):
                if "mask" in file:
                    fetalMask = os.path.join(fetalDir, file)
                elif "template" in file or "recon" in file:
                    fetalTemplate = os.path.join(fetalDir, file)
            outputsFol = None
            outputNum = None
            neonateDir = os.path.join(dataDir, subj, neonateFol)
            for folder in os.listdir(neonateDir):
                if "Outputs" in folder:
                    outputsFol = os.path.join(neonateDir, folder)
                    outputNum = int(folder[7])
                    break
            neonateDir = os.path.join(outputsFol, "T2_Bias_Corrected")
            neonateFile = None
            for file in os.listdir(neonateDir):
                neonateFile = os.path.join(neonateDir, file)
            subjDict[subj] = (fetalTemplate, fetalMask, neonateFile, outputNum)
    return subjDict

In [ ]:
subjDict = findFileLocations()
dataDir = "/data/subjects"
for subj in os.listdir(dataDir):
    if subj.isdigit():
        neonateFol = subj + "_neonate" 
        neonateDir = os.path.join(dataDir, subj, neonateFol)
        count = len([x for x in os.listdir(neonateDir) if not x[0] == '.'])
        if count > 1:
            print(os.listdir(neonateDir))
            print(subj, subjDict[subj][3])
            

In [ ]:
def process():
    subjDict = findFileLocations()
    schedtext = "/data/code/sched.txt"
    for subj in tqdm(subjDict):
        template = subjDict[subj][0]
        mask = subjDict[subj][1]
        neonate = subjDict[subj][2]
        directory = "/data/subjects/" + subj + "/" + subj + "_outputs"
        try:
            if not os.path.exists(directory):
                os.makedirs(directory)
        except OSError:
            print ('Error: Creating directory. ' +  directory)
        os.chdir(directory)
        fileDir = os.path.join(directory, "similar.txt")
        # Apply mask
        appliedMask = getMask(template, mask)
        # Register images and evaluate how similar they are
        flirtFile = getFLIRT(appliedMask, neonate)
        similarity = getSimilarityBetweenVolumes(appliedMask, neonate, schedtext)
        # Creates text file for similiarity value
        with open(fileDir, "w") as f:
            f.write(str(similarity))

In [ ]:
schedtext = "/data/code/sched.txt"
mask = "/data/subjects/0006/0006_fetal/0006_srr_template_mask.nii.gz"
template = "/data/subjects/0006/0006_fetal/0006_srr_template.nii.gz"
neonatal = "/data/subjects/0006/0006_neonate/0006_MR1_Outputs/T2_Bias_Corrected/2-COR_T2_BLADE_2MM_T2_Bias_Corrected.nii.gz"
appliedMask = getMask(template, mask)
simVal = getSimilarityBetweenVolumes(neonatal, appliedMask, schedtext)